In [6]:
!pip install wxPython

In [2]:
import wx
import math 

In [3]:
#variables del suelo y materiales


#inputs fijos
input_Cohesion = None
input_Sigma_adm = None
input_Gamma_suelo = None
input_Phi = None
# JcGG_FLAG: AGREGAR DATO SI ES SUMERGIDO 

#inputs opcionales
inputOP_Betha = None
inputOP_Gamma_concr = None
inputOP_Gamma_suelo_rell = None
inputOP_fy = None
inputOP_fprimac = None

#Calculos
input_kp = None

class MyFrameInputsSuelo(wx.Frame):
    def __init__(self, parent, title):
        super(MyFrameInputsSuelo, self).__init__(parent, title=title, size=(400, 500))  # tamaño de la ventana
        self.panel = wx.Panel(self)
        self.vbox = wx.BoxSizer(wx.VERTICAL)
        self.grid_sizer = wx.FlexGridSizer(4, 2, 10, 10)  # 4 filas, 2 columnas

####        
# INPUTS a grid_sizer
        description1 = wx.StaticText(self.panel, label=f'C\tCohesión [kN/m\u00b2]:')  
        self.textbox1 = wx.TextCtrl(self.panel)
        self.grid_sizer.Add(description1, 0, wx.ALIGN_CENTER_VERTICAL)
        self.grid_sizer.Add(self.textbox1, 0, wx.EXPAND)

        description2 = wx.StaticText(self.panel, label=f'σ_cap\tCapacidad admisible [KPa]:')  
        self.textbox2 = wx.TextCtrl(self.panel)
        self.grid_sizer.Add(description2, 0, wx.ALIGN_CENTER_VERTICAL)
        self.grid_sizer.Add(self.textbox2, 0, wx.EXPAND)

        description3 = wx.StaticText(self.panel, label=f'γ_s\tDensidad del suelo [kN/m\u00b3]')  
        self.textbox3 = wx.TextCtrl(self.panel)
        self.grid_sizer.Add(description3, 0, wx.ALIGN_CENTER_VERTICAL)
        self.grid_sizer.Add(self.textbox3, 0, wx.EXPAND)

        description4 = wx.StaticText(self.panel, label=f'φ\tÁngulo de fricción interna [°]:')  
        self.textbox4 = wx.TextCtrl(self.panel)
        self.grid_sizer.Add(description4, 0, wx.ALIGN_CENTER_VERTICAL)
        self.grid_sizer.Add(self.textbox4, 0, wx.EXPAND)

####
# HIDDEN PANEL
        self.btn_toggle = wx.Button(self.panel, label="▲ Menos Variables Opcionales")
        self.btn_toggle.Bind(wx.EVT_BUTTON, self.on_toggle)
        
        self.hidden_panel = wx.Panel(self.panel)
        self.vbox_hidden = wx.BoxSizer(wx.VERTICAL)
        self.grid_hidden = wx.FlexGridSizer(5, 2, 10, 10)  # 5 filas, 2 columnas
        # self.hidden_panel.Hide()  # Inicialmente está oculto
        self.hidden_panel.Show()

        self.vbox_hidden.Add(self.grid_hidden, 0, flag = wx.ALL | wx.EXPAND, border=10)
        self.hidden_panel.SetSizer(self.vbox_hidden)

####
# Inputs Opcionales
        description_hidden1 = wx.StaticText(self.hidden_panel, label=f'β\tÁngulo de arrancamiento [°]')
        self.hidden_entry1 = wx.TextCtrl(self.hidden_panel)
        self.grid_hidden.Add(description_hidden1, 0, wx.ALIGN_CENTER_VERTICAL)
        self.grid_hidden.Add(self.hidden_entry1, 0, wx.EXPAND)

        description_hidden2 = wx.StaticText(self.hidden_panel, label=f'γ_concr\tDensidad del concreto [kN/m\u00b3]')
        self.hidden_entry2 = wx.TextCtrl(self.hidden_panel)
        self.grid_hidden.Add(description_hidden2, 0, wx.ALIGN_CENTER_VERTICAL)
        self.grid_hidden.Add(self.hidden_entry2, 0, wx.EXPAND)

        description_hidden3 = wx.StaticText(self.hidden_panel, label=f'γ_rell\tDensidad del suelo de relleno [kN/m\u00b3]')
        self.hidden_entry3 = wx.TextCtrl(self.hidden_panel)
        self.grid_hidden.Add(description_hidden3, 0, wx.ALIGN_CENTER_VERTICAL)
        self.grid_hidden.Add(self.hidden_entry3, 0, wx.EXPAND)

        description_hidden4 = wx.StaticText(self.hidden_panel, label=f'fy\tFluencia del acero [MPa]')
        self.hidden_entry4 = wx.TextCtrl(self.hidden_panel)
        self.grid_hidden.Add(description_hidden4, 0, wx.ALIGN_CENTER_VERTICAL)
        self.grid_hidden.Add(self.hidden_entry4, 0, wx.EXPAND)

        description_hidden5 = wx.StaticText(self.hidden_panel, label=f'f\'c\tCapacidad a compresión del concreto [MPa]')
        self.hidden_entry5 = wx.TextCtrl(self.hidden_panel)
        # spin_ctrl_double   = wx.SpinCtrlDouble(parent, value="0.00", min=0.0, max=100.0, inc=0.01)

        self.grid_hidden.Add(description_hidden5, 0, wx.ALIGN_CENTER_VERTICAL)
        self.grid_hidden.Add(self.hidden_entry5, 0, wx.EXPAND)

        # Botones
        self.btn_save = wx.Button(self.panel, label="   OK   ")
        self.btn_save.Bind(wx.EVT_BUTTON, self.on_save)

        # Organización de los elementos en el layout
        self.vbox.Add(self.grid_sizer, 0, flag = wx.ALL | wx.EXPAND, border=10)
        self.vbox.Add(self.btn_toggle, 0, flag = wx.ALIGN_CENTER | wx.TOP, border=10)
        self.vbox.Add(self.hidden_panel, 0, wx.ALL | wx.CENTER, 10)
        self.vbox.Add(self.btn_save, 0, flag = wx.ALIGN_CENTER | wx.TOP, border=10)

        self.panel.SetSizer(self.vbox)

#######################################

    def on_save(self, event):
        # Cargar valores a las entry_boxes
        self.input_Cohesion = float(self.textbox1.GetValue())
        self.input_Sigma_adm = float(self.textbox2.GetValue())
        self.input_Gamma_suelo = float(self.textbox3.GetValue())
        self.input_Phi = float(self.textbox4.GetValue())   
        
        self.inputOP_Betha = float(self.hidden_entry5.GetValue())
        self.inputOP_Gamma_concr = float(self.hidden_entry5.GetValue())
        self.inputOP_Gamma_suelo_rell = float(self.hidden_entry5.GetValue())
        self.inputOP_fy = float(self.hidden_entry5.GetValue())
        self.inputOP_fprimac = float(self.hidden_entry5.GetValue())

        self.input_kp = round(math.tan(math.radians(45 + self.input_Phi/2))**2, 2)

        # print('  ok  ')  
        self.Close()

        
    def on_toggle(self, event):
        self.input_Cohesion = self.textbox1.GetValue()
        self.input_Sigma_adm = self.textbox2.GetValue()
        self.input_Gamma_suelo = self.textbox3.GetValue()
        self.input_Phi = self.textbox4.GetValue()
        
        # SI LAS ENTRADAS TIENEN CONTENIDO, MOSTRAR
        if not "" in [self.input_Cohesion, self.input_Sigma_adm, self.input_Gamma_suelo, self.input_Phi]:
            #Convertir a numero
            self.input_Cohesion = float(self.textbox1.GetValue())
            self.input_Sigma_adm = float(self.textbox2.GetValue())
            self.input_Gamma_suelo = float(self.textbox3.GetValue())
            self.input_Phi = float(self.textbox4.GetValue())

            self.inputOP_Betha = round(2/3*self.input_Phi,2)
            self.inputOP_Gamma_concr = 24
            self.inputOP_Gamma_suelo_rell = self.input_Gamma_suelo
            self.inputOP_fy = 420
            self.inputOP_fprimac = 21

            #Convertir a txt 
            self.hidden_entry1.SetValue(str(self.inputOP_Betha))
            self.hidden_entry2.SetValue(str(self.inputOP_Gamma_concr))
            self.hidden_entry3.SetValue(str(self.inputOP_Gamma_suelo_rell))
            self.hidden_entry4.SetValue(str(self.inputOP_fy))
            self.hidden_entry5.SetValue(str(self.inputOP_fprimac))            
            
            if self.hidden_panel.IsShown():
                self.hidden_panel.Hide()
                self.btn_toggle.SetLabel("▼ Más Variables Opcionales")

            else:
                self.hidden_panel.Show()
                self.btn_toggle.SetLabel("▲ Menos Variables Opcionales")
        else:
            self.btn_toggle.SetLabel("▼ *INGRESAR INPUTS* ▼")
        # Reajustar la interfaz
        self.Layout()

    def init_toggle(self, event):
        self.hidden_panel.Hide()
        self.btn_toggle.SetLabel("▼ Más Variables Opcionales")    
        

# Clase principal de la aplicación que hereda de wx.App
class VentanaInputsSuelo(wx.App):
    def OnInit(self):
        # Crear una instancia de la ventana
        self.frame = MyFrameInputsSuelo(None, "Variables input")
        # Mostrar la ventana
        self.frame.Show(True)
        
        # Ejecutar el toggle programáticamente después de que la ventana se haya mostrado
        wx.CallAfter(self.frame.init_toggle, None)  # Llamamos a on_toggle() para ocultar el panel

        return True

# Ejecutar la aplicación
if __name__ == "__main__":
    app = VentanaInputsSuelo(False)
    app.MainLoop()

In [ ]:
# Reacciones matriz
import wx
import wx.grid

f_diag_ort_casos = None
# JcGG_FLAG: MODIFICAR EL ORDEN PARA QUE SALGA IGUAL AL EXCEL

class MyFrameReaccionesDiagOrt(wx.Frame):
    def __init__(self, parent, title):
        super(MyFrameReaccionesDiagOrt, self).__init__(parent, title=title, size=(650, 250))
        
        self.panel_reacciones = wx.Panel(self)
        self.grid_matrix = wx.FlexGridSizer(2, 2, 0, 0)

        #CREAR ETIQUETAS
        self.lab_diag = wx.StaticText(self.panel_reacciones, label = f'Sistema Diagonal [kN]')
        self.lab_ort = wx.StaticText(self.panel_reacciones, label = f'Sistema Ortogonal [kN]')
        
        #GRIDS
        self.grid_diag = wx.grid.Grid(self.panel_reacciones)
        self.grid_diag.CreateGrid(4, 3) 

        self.grid_ort = wx.grid.Grid(self.panel_reacciones)
        self.grid_ort.CreateGrid(4, 3)

        #TAMAÑO DE LAS CELDAS
        for col in range(self.grid_diag.GetNumberCols()):
            self.grid_diag.SetColSize(col, 80)  # Ajusta el tamaño de las columnas a 100 píxeles
        for row in range(self.grid_diag.GetNumberRows()):
            self.grid_diag.SetRowSize(row, 25)  # Ajusta el tamaño de las filas a 30 píxeles

        for col in range(self.grid_ort.GetNumberCols()):
            self.grid_ort.SetColSize(col, 80)  # Ajusta el tamaño de las columnas a 100 píxeles
        for row in range(self.grid_ort.GetNumberRows()):
            self.grid_ort.SetRowSize(row, 25)  # Ajusta el tamaño de las filas a 30 píxeles  
            
        #BLOQUEAR EL AJUSTE DE TAMÑOS
        self.grid_diag.EnableDragColSize(False)  # Desactiva la capacidad de redimensionar las columnas
        self.grid_diag.EnableDragRowSize(False)

        #UBICACIÓN EN grid_matrix
        self.grid_matrix.Add(self.lab_diag, wx.ALIGN_CENTER_VERTICAL)
        self.grid_matrix.Add(self.lab_ort, wx.ALIGN_CENTER_VERTICAL)
        
        self.grid_matrix.Add(self.grid_diag, wx.ALIGN_CENTER_VERTICAL)
        self.grid_matrix.Add(self.grid_ort, wx.ALIGN_CENTER_VERTICAL)

        
        # ETIQUETAS grid_matrix
        # self.grid_diag.SetColLabelValue(0, "CASO DE CARGA")
        self.grid_diag.SetColLabelValue(0, "FV")
        self.grid_diag.SetColLabelValue(1, "FL")
        self.grid_diag.SetColLabelValue(2, "FT")
        #
        self.grid_diag.SetRowLabelValue(0, "Compresión")
        self.grid_diag.SetRowLabelValue(1, "Tracción")
        self.grid_diag.SetRowLabelValue(2, "Max Longitudinal")
        self.grid_diag.SetRowLabelValue(3, "Max Transversal")

        self.grid_ort.SetColLabelValue(0, "FV")
        self.grid_ort.SetColLabelValue(1, "FL")
        self.grid_ort.SetColLabelValue(2, "FT")   
        #
        self.grid_ort.HideRowLabels()

        #BOTON DE GUARDADO
        self.btn_reacciones = wx.Button(self.panel_reacciones, label = '   OK   ')
        self.btn_reacciones.Bind(wx.EVT_BUTTON, self.save_reacciones)
        
        # CONFIGURAR EL LAYOUT vbox
        self.vbox = wx.BoxSizer(wx.VERTICAL)
        self.vbox.Add(self.grid_matrix, 1, flag = wx.EXPAND | wx.ALL, border = 5)
        self.vbox.Add(self.btn_reacciones, 0, flag = wx.ALIGN_CENTER | wx.TOP)

        
        self.panel_reacciones.SetSizerAndFit(self.vbox)
        self.Show()


    def save_reacciones(self, event):
        lista_reacciones_diag = [self.grid_diag.GetCellValue(f, c) for f in range(self.grid_diag.GetNumberRows()) for c in range(self.grid_diag.GetNumberCols())]
        lista_reacciones_ort = [self.grid_ort.GetCellValue(f, c) for f in range(self.grid_ort.GetNumberRows()) for c in range(self.grid_ort.GetNumberCols())]

        valores_reacciones_diag = [(0 if i == "" else float(i)) for i in lista_reacciones_diag]
        valores_reacciones_ort = [(0 if i == "" else float(i)) for i in lista_reacciones_ort]

        fv_diag_comp, fl_diag_comp, ft_diag_comp, fv_diag_tr, fl_diag_tr, ft_diag_tr, fv_diag_long, fl_diag_long, ft_diag_long, fv_diag_trnsv, fl_diag_trnsv, ft_diag_trnsv = valores_reacciones_diag
        fv_ort_comp, fl_ort_comp, ft_ort_comp, fv_ort_tr, fl_ort_tr, ft_ort_tr, fv_ort_long, fl_ort_long, ft_ort_long, fv_ort_trnsv, fl_ort_trnsv, ft_ort_trnsv = valores_reacciones_ort
        
        self.Close()

        
# Clase principal de la aplicación que hereda de wx.App
class VentanaDeReacciones(wx.App):
    def OnInit(self):
        self.frame = MyFrameReaccionesDiagOrt(None, "Variables input")
        self.frame.Show(True)
        return True

# Ejecutar la aplicación
if __name__ == "__main__":
    app = VentanaDeReacciones(False)
    app.MainLoop()